In [1]:
import numpy as np
import ccxt
from ccxt.base.errors import InsufficientFunds
import pandas as pd
import schedule 
import time
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import ssl
import json
from songline import Sendline

In [2]:
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [3]:
cap = 36000
ceil = 10180 ##new
floor = 2500 ##new
gap = 120
level = 64 ## (ceil-floor)/gap
cost = 562.5 ## cap/level
pair = 'ETHBULL/USD'
symbol = 'ETHBULL'

In [4]:
api_key = 'xxx' # API Keys
secret = 'xx' # API Secret
subaccount = 'x'  # Choose sub-acc

ftx = ccxt.ftx({
    'api_key': api_key,
    'secret': secret,
    'enableRateLimit': True, 'rateLimit': 2000
})

ftx.headers = {'FTX-SUBACCOUNT': subaccount,}
token_line = 'x'
messenger = Sendline(token_line)

In [7]:
layerss = [x for x in range(floor, ceil+gap, gap)]  
layers = [x for x in range(floor, ceil, gap)]  
zip_amount = dict(zip(layers, [round(cost/x, 4) for x in layers]) )

### # Only First Setup

In [28]:
# last_price = ftx.fetch_ticker(pair)['last']
# first_orders = list( filter(lambda x: x< last_price, layers) )
# addup_orders = list( filter(lambda x: x> last_price, layers) )

In [29]:
# ## for first run half bot ## buy limit
# for i in first_orders:
#     ftx.create_order(pair,'limit','buy', round(cost/i,4), i)

In [30]:
# ## for first run half top ## buy market
# addup_fund = len(addup_orders)*cost    ##จน.ชั้น*ทุนต่อชั้น
# amount_addup = (  cost*len(addup_orders)/min(addup_orders)  )/ len(addup_orders)  ##จน.หุ้นที่ต้องวางขายแต่ละชั้น

# ftx.create_market_buy_order(pair, round(addup_fund/last_price,4) )   ##ซื้อรวบ

In [31]:
## then sell limit
# amount_addup = (  cost*len(addup_orders)/min(addup_orders)  )/ len(addup_orders)
# for i in addup_orders:
#     ftx.create_order(pair,'limit','sell', round(amount_addup,4), i+gap)   ##วางขายแต่ละชั้นเท่า ๆกัน

In [32]:
# def revise_zone():  
#     addup_fund = len(addup_orders)*cost
#     current_amount = ftx.fetch_balance()['total'][symbol]
#     total_amount = addup_fund/last_price
#     lack_amount = total_amount - current_amount
    
# #     if lack_amount > 0 :
# #         ftx.create_market_buy_order(pair, lack_amount) 

#     return lack_amount
# revise_zone()

In [33]:
# def ls_priced():
#     global layerz
#     priced = int(ftx.fetchMyTrades(pair)[-1]['price'])
#     low_bound = priced-(3*gap)
#     up_bound = priced+(3*gap)
#     layerz = [x for x in range(low_bound, up_bound, gap)]   ## ใช้แค่นี้ เมื่อมี cf จะเติมซื้อทันทีที่หลุดโซนล่าง
    
#     if low_bound < floor:  ## ใช้ เมื่อหลุดโซนไม่ได้ (ไม่มีเงินสำรองเติม)
#         layerz = [x for x in range(floor, up_bound, gap)]
#     elif up_bound > ceil:
#         layerz = [x for x in range(low_bound, ceil, gap)]

In [34]:
# def ggs_insert(df):
#     worksheet.insert_row([df.iloc[-1]['datetime'], df.iloc[-1]['side'], df.iloc[-1]['price'], 
#     round(balance['info']['result'][0]['usdValue']+balance['info']['result'][2]['usdValue'],4),
#     round(balance['free']['USD'],4) ], 2)

In [35]:
def missing_layers():
    
    global missing_list, od_list
    orders = ftx.fetch_open_orders(pair)
    missing_list = [x for x in layerss if x not in [i['price'] for i in orders]]
    od_list = missing_list[:-1]

In [36]:
def regular_open():

    for i in od_list:
    
        lp = {int(x['price']):x['id'] for x in ftx.fetchMyTrades(limit = 34)}

        if (lp[i] > lp[i+gap]):       ## แสดงว่าชั้นล่างเกิดก่อน (fresh buy)
            ftx.create_order(pair,'limit','sell', zip_amount[i], i+gap )

        else:     
            ftx.create_order(pair,'limit','buy', zip_amount[i], i)
        

In [37]:
def first_stage():
    
    for i in od_list:
    
        if ftx.fetch_my_trades()[-1]['side'] == 'buy':
            ftx.create_order(pair,'limit','sell', zip_amount[i], i+gap )
            
        else:
            ftx.create_order(pair,'limit','buy', zip_amount[i], i)
            
        global stop
        stop = True

In [23]:
def recover_holes3():
    
    a = ftx.fetch_my_trades()
    b = {i['price']: [int(i['id']), i['side']] for i in a if i['price'] in missing_list}
    del b[min(b, key=b.get)] ## delete disqualified candidate
    
    for pricE, sidE in b.items() :
        
        if sidE[1] == 'buy' :
            
                last_price = ftx.fetch_ticker(pair)['last']

                if last_price > pricE+gap :
                    
                    ftx.create_market_sell_order(pair, zip_amount[pricE])  #market
                    ftx.create_order(pair,'limit','buy', zip_amount[pricE], pricE) #took
                    
                else:
                    
                    ftx.create_order(pair,'limit','sell', zip_amount[pricE], pricE+gap ) #took
                                    
        elif sidE[1] == 'sell' :
            
                last_price = ftx.fetch_ticker(pair)['last']

                if last_price < pricE-gap :
                    
                    ftx.create_market_buy_order(pair, zip_amount[pricE-gap])  #market
                    ftx.create_order(pair,'limit','sell', zip_amount[pricE-gap], pricE) #took

                else:
                    
                    ftx.create_order(pair,'limit','buy', zip_amount[pricE-gap], pricE-gap) #took


In [26]:
def first_stages():
    try:
        
        missing_layers()
        
        if len(missing_list) == 2:      
            
            first_stage()    
            print('first stage done!')
            
    except InsufficientFunds as e:
        global go
        print('ff')
        go = False
        time.sleep(1)
    except ccxt.NetworkError as e:
        print(ftx.id, 'fetch_order_book failed due to a network error:', str(e))
        time.sleep(1)
        check_orders()
    except ccxt.ExchangeError as e:
        print(ftx.id, 'fetch_order_book failed due to exchange error:', str(e))
        time.sleep(1)
        check_orders()
    except Exception as e:
        print(ftx.id, 'fetch_order_book failed with:', str(e))
        time.sleep(1)
        check_orders()

In [27]:
def check_orders():
    
    try:
        
        missing_layers()
        
        if len(missing_list) == 2:
                            
            regular_open()
        
        elif len(missing_list) > 2:
            
            recover_holes3()
            
    except InsufficientFunds as e:
        global go
        go = False
        messenger.sendtext('check bot pls, x')
        print(ftx.id, str(e))
    except ccxt.NetworkError as e:
        print(ftx.id, 'fetch_order_book failed due to a network error:', str(e))
        time.sleep(1)
        check_orders()
    except ccxt.ExchangeError as e:
        print(ftx.id, 'fetch_order_book failed due to exchange error:', str(e))
        time.sleep(1)
        check_orders()
    except Exception as e:
        print(ftx.id, 'fetch_order_book failed with:', str(e))
        time.sleep(1)
        check_orders()

## check date if buy or sell come first -> do in the other way

In [28]:
# schedule.every(0.1).seconds.do(check_orders)
go = True
while go:
#     schedule.run_pending()
#     if len(layers) - len(ftx.fetch_open_orders()) > 2:
#         break
    check_orders()
    time.sleep(1)

52400.0 buy
53200.0 buy
ftx ftx {"success":false,"error":"Not enough balances"}



### #For redundancy check

In [8]:
df0 = pd.DataFrame(ftx.fetch_open_orders(pair),columns=['datetime', 'price', 'amount', 'side']) 

def check_dup(df):
    dfdup = df[df.duplicated(['price'])]
    return dfdup

In [9]:
def sort_orders(df):
    global dfsort
    dfsort = df.sort_values(by=['price'],  ascending=False)
    pd.set_option('display.max_rows', dfsort.shape[0]+1)
    return dfsort

In [10]:
def list_check(df):

    return print('{}{}{}{}'.format('lack',np.setdiff1d(layers, df['price'].tolist() ),
                                   'over',np.setdiff1d(df['price'].tolist(), layers ) ) )

In [11]:
len(ftx.fetch_open_orders(pair)) < len(layers)

True

In [12]:
check_dup(df0)

,datetime,price,amount,side


In [13]:
list_check(df0)

lack[25200 26000 26800 27600 28400 29200 30000 30800 31600 32400 33200 34000
 34800 35600 36400 37200 38000 38800 39600 40400 41200 42000 42800 43600
 44400 45200 46000 46800 47600 48400 49200 50000 50800 51600 52400 53200
 54000 54800 55600 56400 57200 58000 58800 59600 60400 61200 62000 62800
 63600 64400 65200 66000 66800 67600 68400 69200 70000 70800 71600 72400
 73200 74000 74800 75600 76400 77200 78000 78800 79600 80400 81200 82000
 82800 83600 84400 85200 86000 86800 87600 88400 89200 90000 90800]over[ 3500.  3750.  4000.  4250.  4500.  4750.  5000.  5250.  5500.  5750.
  6000.  6250.  6750.  7000.  7250.  7500.  7750.  8000.  8250.  8500.
  8750.  9000.  9250.  9500.  9750. 10000. 10250. 10500. 10750. 11000.
 11250. 11500. 11750. 12000. 12250. 12500. 12750. 13000. 13250. 13500.
 13750. 14000. 14250. 14500. 14750.]


In [14]:
sort_orders(df0)

,datetime,price,amount,side
2,2021-05-22T01:09:16.988Z,14750.0,0.02143,sell
1,2021-05-22T01:09:02.021Z,14500.0,0.01844,sell
0,2021-05-22T01:07:29.766Z,14250.0,0.02352,sell
9,2021-09-07T04:04:45.203Z,14000.0,0.01820,sell
10,2021-09-07T07:09:51.399Z,13750.0,0.01850,sell
11,2021-09-07T08:23:54.044Z,13500.0,0.01890,sell
12,2021-09-07T08:29:57.020Z,13250.0,0.01920,sell
13,2021-09-07T08:38:02.505Z,13000.0,0.01960,sell
14,2021-09-07T09:42:07.244Z,12750.0,0.02000,sell
15,2021-09-07T13:52:07.118Z,12500.0,0.02040,sell


In [15]:
len(ftx.fetch_open_orders(pair))

45